In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
df_products_detailed = pd.read_csv("products_detailed.csv")
df_products_detailed.head()

,id,name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,is_natural_organic,is_natural_sephora,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_name,brand_id,brand_long
0,P448857,Apple Cider Vinegar Detangling Conditioner,"An ultra-lightweight conditioner that gently detangles hair and uses the power of apple cider vinegar to seal the hair’s cuticle, enhancing softness and shine.","What it is: An ultra-lightweight conditioner that gently detangles hair and uses the power of apple cider vinegar to seal the hair’s cuticle, enhancing softness and shine. Hair Type: Straight, Wavy, Curly, and Coily Hair Texture: Fine, Medium, and Thick Key Benefits: Detangles, Adds Shine, and Prevents Damage Formulation: Liquid Highlighted Ingredients: - Sunflower Seed Extract: Protects hair color. - Argan Oil, Aloe Vera, and Lavender Extract: Add softness and shine. Ingredient Callouts: Free of sulfates SLS and SLES, parabens, and phthalates. This product is also vegan, cruelty-free, and gluten-free, and comes in recyclable packaging. What Else You Need to Know: This conditioner is formulated to detangle hair and provide an extra conditioning boost after cleansing hair. It is infused with conditioning agents argan oil, aloe vera, and lavender extract to nurture hair and sunflower seed extract to protect hair color.",2276269,6.5 oz/ 192 mL,30.00,None,NaN,30.00,30.00,0,0,0,2.80,5,280,"{'cat60131': 'Conditioner', 'cat1230050': 'Shampoo & Conditioner', 'cat130038': 'Hair'}",https://www.sephora.com/product/apple-cider-vinegar-detangling-conditioner-P448857?skuId=2276269,https://www.sephora.com/productimages/sku/s2276269-main-zoom.jpg,"-Sunflower Seed Extract: Protects hair color. -Argan Oil, Aloe Vera, and Lavender Extract: Add softness and shine. Water/Aqua/Eau, Cetearyl Alcohol, Caprylic/Capric Triglyceride, Behentrimonium Chloride, Cetrimonium Chloride, Stearalkonium Chloride, Diheptyl Succinate, Capryloyl Glycerin/Sebacic Acid Copolymer, Polyquaternium-47, Hydrolyzed Vegetable Protein, Apple Cider Vinegar, Glycerin, Hydroxyethylcellulose, Argania Spinosa Kernel Oil, Macadamia Ternifolia Seed Oil, Spathodea Campanulata Flower Extract, Taraxacum Officinale (Dandelion) Leaf Extract, Lavandula Angustifolia (Lavender) Extract, Aloe Barbadensis Leaf Juice, Tocopheryl Acetate, Butylene Glycol, Helianthus Annuus (Sunflower) Seed Extract, Tetrasodium Glutamate Diacetate, Sodium Hydroxide, Phenoxyethanol, Ethylhexylglycerin, Fragrance (Parfum).","Suggested Usage: -Apply to clean, wet hair. -Massage through mid-lengths and ends of hair, then rinse thoroughly. -For best results, cleanse before with dpHUE Apple Cider Vinegar Hair Rinse.",dpHUE,6147,NaN
1,P425464,Surf Spray Mini,"A saltwater-based styling spray that adds texture, volume, and hold for matte waves that mimic a fresh-off-the-beach look.","What it is: A saltwater-based styling spray that adds texture, volume, and hold for matte waves that mimic a fresh-off-the-beach look. Hair Type: Straight and Wavy Hair Texture: Fine and Medium Hair Concerns: Hold and Volumizing Formulation: Spray Highlighted Ingredients: - Ascophyllum Nudosum (Seaweed) Extract: Extracted from naturally occurring seaweed; softens and smoothes skin and moisturizes hair, increasing its elasticity, luster, and softness; and helps decrease hair’s static charge, thus giving more body and less flyaways. - Macrocystis Pyrifera (Kelp) Extract: A natural seaweed extract with a high iodine and mineral content; contributes shine and manageability to hair and reduces the harshness of some chemicals; the nutritive value of this extract makes it ideal for skin preparations. - Magnesium Sulfate: Commonly known as Epsom salts; a natural mineral, which imparts astringent action. Ingredient Callouts: Free of parabens, formaldehydes, phthalates, and mineral oil. This product is also cruelty-free. What Else You Need to Know: This saltwater-based, st

In [3]:
df_products_detailed.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2898 entries, 0 to 2897
Data columns (total 25 columns):
id                    2898 non-null object
name                  2898 non-null object
short_desc            2898 non-null object
long_desc             2898 non-null object
item                  2898 non-null int64
item_name             2525 non-null object
list_price            2898 non-null float64
variation             2898 non-null object
variants              1283 non-null object
price_low             2898 non-null float64
price_high            2898 non-null float64
is_limited_edition    2898 non-null int64
is_natural_organic    2898 non-null int64
is_natural_sephora    2898 non-null int64
rating                2857 non-null float64
review_count          2898 non-null int64
user_favorites        2898 non-null int64
categories            2896 non-null object
url                   2898 non-null object
image_url             2898 non-null object
ingredients           2473 non-null

In [4]:
categories = []
for group in df_products_detailed[df_products_detailed.categories.isna()==False].categories.unique():
    group = json.loads(group.replace("\'", "\""))
    for key, val in group.items():
        if key:
            categories.append(val)
            
categories = list(set(categories))
len(categories), categories

(153,
 ['Shampoo',
  'Best For',
  'Decollete & Neck Creams',
  'BB & CC Cream',
  'Cologne Gift Sets',
  'Skincare Sets',
  'Hair Styling Products',
  'Color Care',
  'Anti-Aging',
  'Cheek',
  'Bath & Shower',
  'Sponges & Applicators',
  'Treatments',
  'High Tech Tools',
  'Eye Care',
  'Tools & Brushes',
  'Liquid Lipstick',
  'Hair Tools',
  'Toners',
  'Perfume',
  'Cellulite & Stretch Marks',
  'Moisturizers',
  'Eyeliner',
  'Sheet Masks',
  'Eyebrow',
  'Lip',
  'Hair Removal & Shaving',
  'Men',
  'Bath & Body',
  'Bath Soaks & Bubble Bath',
  'Hair Dryers',
  'Fragrance',
  'Hair Primers',
  'Lip Balm & Treatment',
  'Lip Brushes',
  'Makeup & Travel Cases',
  'Lipstick',
  'Nail',
  'Moisturizer & Treatments',
  'Tinted Moisturizer',
  'Holistic Wellness',
  'Candles',
  'Eye Primer',
  'Eye Masks',
  'For Face',
  'Tweezers & Eyebrow Tools',
  'Facial Peels',
  'Hand Cream & Foot Cream',
  'Bronzer',
  'Curling Irons',
  'Body Care',
  'Skincare',
  'Blush',
  'Cleansing 

In [5]:
df_category = df_products_detailed[["id", "categories"]].copy()

for i in range(len(df_category)):
    if df_category.categories.iloc[i] is np.nan:
        continue
    else:
        categories = json.loads(df_category.categories.iloc[i].replace("\'", "\""))
        for cat in categories.values():
            df_category.at[i, cat] = 1
        
df_category

,id,categories,Conditioner,Shampoo & Conditioner,Hair,Mini Size,Moisturizers,Skincare,Rollerballs & Travel Size,Women,Fragrance,Bath & Shower,Men,Facial Cleansing Brushes,High Tech Tools,Perfume,Cellulite & Stretch Marks,Body Care,Bath & Body,Shampoo,Eye Primer,Eye,Makeup,Perfume Gift Sets,Value & Gift Sets,Face Primer,Face,Concealer,Lipstick,Lip,Body Wash & Shower Gel,Hair Styling Products,Hair Styling & Treatments,Lip Gloss,Scrub & Exfoliants,Face Masks,Masks,Blush,Cheek,Skincare Sets,Beauty Supplements,Wellness,Lip Sets,Face Wash & Cleansers,Cleansers,Face Sets,Eye Creams & Treatments,Eye Care,Highlighter,Lip Balm & Treatment,Leave-In Conditioner,Mascara,Foundation,Nail,Eyeliner,Shaving,BB & CC Cream,Dry Shampoo,Lotions & Oils,Bronzer,Face Serums,Treatments,Hair Masks,Face Sunscreen,Sun Care,Face Brushes,Brushes & Applicators,Hair Accessories,Hair Tools,Lip Liner,Cologne,Tinted Moisturizer,Deodorant & Antiperspirant,Anti-Aging,Hair Primers,Eye Palettes,Body Lotions & Body Oils,Body Moisturizers,Hand Cream & Foot Cream,Brush Sets,Exfoliators,Scalp & Hair Treatments,Holistic Wellness,Blemish & Acne Treatments,Eye Cream,Toners,Setting Spray & Powder,Moisturizer & Treatments,Lip Balms & Treatments,Lip Treatments,False Eyelashes,Hair Oil,Sheet Masks,Face Oils,Candles,Candles & Home Scents,Facial Peels,Face Wash,Eyebrow,Facial Rollers,Brush Cleaners,Tweezers & Eyebrow Tools,Accessories,Tools & Brushes,Eyeshadow,Sponges & Applicators,Color Correct,Aftershave,Hair Spray,Contour,Liquid Lipstick,Makeup Removers,Night Creams,Self Tanners,Decollete & Neck Creams,Mists & Essences,Sunscreen,Mirrors & Sharpeners,Eye Masks,Body Mist & Hair Mist,Hair Thinning & Hair Loss,Hair Straighteners & Flat Irons,Cologne Gift Sets,Eye Brushes,Curling Irons,Body Products,Other Needs,Lip Brushes,Teeth Whitening,Bath Soaks & Bubble Bath,Makeup Bags & Travel Cases,For Face,Makeup Palettes,Color Care,Best For,Cheek Palettes,Hair Dryers,For Body,Hair Products,BB & CC Creams,Hair Brushes & Combs,Body Sunscreen,Cleansing Brushes,Eye Sets,Hand Wash & Soap,Body Sprays & Deodorant,Lip Stain,Hair Removal,Face Wipes,Hair Removal & Shaving,Curls & Coils,Deodorant for Men,Eyelash Curlers,Diffusers,Makeup & Travel Cases
0,P448857,"{'cat60131': 'Conditioner', 'cat1230050': 'Shampoo & Conditioner', 'cat130038': 'Hair'}",1.00,1.00,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,P425464,"{'cat1910033': 'Hair', 'cat1830032': 'Mini Size'}",nan,nan,1.00,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,P414293,"{'cat60097': 'Moisturizers', 'cat1230034': 'Moisturizers', 'cat150006': 'Skincare'}",nan,nan,nan,nan,1.00,1.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

In [6]:
df_category.drop(columns="categories", axis=1, inplace=True)
df_category.fillna(0, inplace=True)

In [7]:
for col in df_category.columns[1:]:
    df_category[col] = df_category[col].astype("int64")
df_category  

,id,Conditioner,Shampoo & Conditioner,Hair,Mini Size,Moisturizers,Skincare,Rollerballs & Travel Size,Women,Fragrance,Bath & Shower,Men,Facial Cleansing Brushes,High Tech Tools,Perfume,Cellulite & Stretch Marks,Body Care,Bath & Body,Shampoo,Eye Primer,Eye,Makeup,Perfume Gift Sets,Value & Gift Sets,Face Primer,Face,Concealer,Lipstick,Lip,Body Wash & Shower Gel,Hair Styling Products,Hair Styling & Treatments,Lip Gloss,Scrub & Exfoliants,Face Masks,Masks,Blush,Cheek,Skincare Sets,Beauty Supplements,Wellness,Lip Sets,Face Wash & Cleansers,Cleansers,Face Sets,Eye Creams & Treatments,Eye Care,Highlighter,Lip Balm & Treatment,Leave-In Conditioner,Mascara,Foundation,Nail,Eyeliner,Shaving,BB & CC Cream,Dry Shampoo,Lotions & Oils,Bronzer,Face Serums,Treatments,Hair Masks,Face Sunscreen,Sun Care,Face Brushes,Brushes & Applicators,Hair Accessories,Hair Tools,Lip Liner,Cologne,Tinted Moisturizer,Deodorant & Antiperspirant,Anti-Aging,Hair Primers,Eye Palettes,Body Lotions & Body Oils,Body Moisturizers,Hand Cream & Foot Cream,Brush Sets,Exfoliators,Scalp & Hair Treatments,Holistic Wellness,Blemish & Acne Treatments,Eye Cream,Toners,Setting Spray & Powder,Moisturizer & Treatments,Lip Balms & Treatments,Lip Treatments,False Eyelashes,Hair Oil,Sheet Masks,Face Oils,Candles,Candles & Home Scents,Facial Peels,Face Wash,Eyebrow,Facial Rollers,Brush Cleaners,Tweezers & Eyebrow Tools,Accessories,Tools & Brushes,Eyeshadow,Sponges & Applicators,Color Correct,Aftershave,Hair Spray,Contour,Liquid Lipstick,Makeup Removers,Night Creams,Self Tanners,Decollete & Neck Creams,Mists & Essences,Sunscreen,Mirrors & Sharpeners,Eye Masks,Body Mist & Hair Mist,Hair Thinning & Hair Loss,Hair Straighteners & Flat Irons,Cologne Gift Sets,Eye Brushes,Curling Irons,Body Products,Other Needs,Lip Brushes,Teeth Whitening,Bath Soaks & Bubble Bath,Makeup Bags & Travel Cases,For Face,Makeup Palettes,Color Care,Best For,Cheek Palettes,Hair Dryers,For Body,Hair Products,BB & CC Creams,Hair Brushes & Combs,Body Sunscreen,Cleansing Brushes,Eye Sets,Hand Wash & Soap,Body Sprays & Deodorant,Lip Stain,Hair Removal,Face Wipes,Hair Removal & Shaving,Curls & Coils,Deodorant for Men,Eyelash Curlers,Diffusers,Makeup & Travel Cases
0,P448857,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,P425464,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,P414293,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,P377493,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,P425872,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [8]:
df_category.to_csv("category.csv", index=False)